# mlflow Reproducibility

This notebook demonstrates mlflow's capability to reproduce results from experimental runs.

By providing the unique identifier for a run, this notebook will retrieve the model and training and test data sets for that run, retrain the model on the training data and calculate the metric score on the test data.

In [1]:
from __future__ import print_function

In [2]:
import os
import os.path
import socket
import pandas as pd
import tempfile
import pickle
import shutil
import scipy.stats
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn
import mlflow.tracking


## Set up connection to mlflow tracking server

In [3]:
# Assumes MFLOW_TRACKING_URI is set

client = mlflow.tracking.MlflowClient()
mlflow.get_tracking_uri()

'http://mlflow_tracker:5000'

In [4]:
for e in client.list_experiments():
    print('experiement:', e.name)

experiement: Default
experiement: mlflow_demo1
experiement: mlflow_demo2


## Specify Run identifier to retrieve model and data

In [5]:
RUN_UUID = '4a69365821b94b5da2a77d2f914d4861'

## Retrieve training/test data sets from run

In [6]:
# retrieve pickled data file
data_dir = client.download_artifacts(RUN_UUID,'data_sets')
print("contents of data download directory",data_dir,os.listdir(data_dir))


contents of data download directory /tmp/tmpmqjv5zb4/data_sets ['data.pkl']


In [7]:
os.path.join(data_dir,os.listdir(data_dir)[0])

'/tmp/tmpmqjv5zb4/data_sets/data.pkl'

In [8]:
# unpickle training and test data
with open(os.path.join(data_dir,os.listdir(data_dir)[0]),'rb') as f:
    data = pickle.load(f)

In [9]:
data['train_x'][:5]

array([[7.2000e+00, 2.4000e-01, 2.4000e-01, 1.7000e+00, 4.5000e-02,
        1.8000e+01, 1.6100e+02, 9.9196e-01, 3.2500e+00, 5.3000e-01,
        1.1200e+01],
       [7.6000e+00, 3.3000e-01, 4.1000e-01, 1.3700e+01, 4.5000e-02,
        4.4000e+01, 1.9700e+02, 9.9890e-01, 3.1800e+00, 6.4000e-01,
        9.1000e+00],
       [7.0000e+00, 2.7000e-01, 3.2000e-01, 6.8000e+00, 4.7000e-02,
        4.7000e+01, 1.9300e+02, 9.9380e-01, 3.2300e+00, 3.9000e-01,
        1.1400e+01],
       [7.2000e+00, 2.0000e-01, 3.4000e-01, 2.7000e+00, 3.2000e-02,
        4.9000e+01, 1.5100e+02, 9.9000e-01, 3.1600e+00, 3.9000e-01,
        1.2700e+01],
       [5.7000e+00, 2.5500e-01, 6.5000e-01, 1.2000e+00, 7.9000e-02,
        1.7000e+01, 1.3700e+02, 9.9307e-01, 3.2000e+00, 4.2000e-01,
        9.4000e+00]])

## Retrieve Model

In [10]:
# retrieve pickled data file
model_dir = client.download_artifacts(RUN_UUID,'best_estimator')
print("contents of model download directory",model_dir,os.listdir(model_dir))


contents of model download directory /tmp/tmphxljcbns/best_estimator ['MLmodel', 'conda.yaml', 'model.pkl']


In [11]:

sklearn_model = mlflow.sklearn.load_model(model_dir)
sklearn_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.058512491882074746, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, n_estimators=453, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=13, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

## Train model on training data set

In [12]:
sklearn_model.fit(data['train_x'],data['train_y'].quality)

/opt/conda/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.058512491882074746, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, n_estimators=453, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=13, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

## Caclulate score on test data set

In [13]:
sklearn_model.score(data['test_x'],data['test_y'].quality)

0.5003647695314799

## Retrieve saved metric from training run

In [14]:
run_info = client.get_run(RUN_UUID)
for m in run_info.data.metrics:
    print(m.key, m.value)

r2 0.50036476953148


## Clean up

In [15]:
shutil.rmtree(data_dir)
shutil.rmtree(model_dir)